In [3]:
from rdkit import Chem, RDLogger
from rdkit.Chem import AllChem, RDConfig, Descriptors, rdChemReactions
RDLogger.DisableLog("rdApp.*")

import pandas as pd
import numpy as np
from pathlib import Path

# SAScore, LogP, cycles, QED import
from rdkit.Chem.Crippen import MolLogP, MolMR
#from SAS_calculator.sascorer import calculateScore
from rdkit.Chem.Descriptors import qed
import networkx as nx

import olympus
from olympus.utils.misc import get_hypervolume, get_pareto, get_pareto_set
#from janus_olympus import create_value_space, create_scalarizer, scalarize_and_sort
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tqdm import tqdm

In [4]:
def top_min_all(optim_type):
    pwd = "/Users/nkusanda/Desktop/RESEARCH/olympus-ai4mat/DATA_3opt/RESULTS_"
    min_distances = []

    min_qed = []
    min_logp = []
    min_sas = []
    min_dist = []
    min_smi = []

    for i in tqdm(range(1,6)):
        
        if i == 1:
            df = pd.read_csv(pwd + optim_type + "/smiles_collector.csv")
        else:
            df = pd.read_csv(pwd + optim_type + str(i) + "/smiles_collector.csv")


        smiles = list(df['smi'])
        logp = list(df['logp'])
        qed = list(df['qed'])
        sas = list(df['sas'])

        data = np.column_stack((qed,logp,sas))
        utopia = np.array([0.6,10,0.5])
        dist_1 = (utopia[0] - data[:,0]) / utopia[0]
        dist_2 = (utopia[1] - data[:,1]) / utopia[1]
        dist_3 = (utopia[2] - data[:,2]) / utopia[2]

        dist_1 = np.array([dist_1[i] if dist_1[i]>0 else 0 for i in range(len(dist_1))])
        dist_2 = np.array([dist_2[i] if dist_2[i]>0 else 0 for i in range(len(dist_2))])
        dist_3 = np.array([dist_3[i] if dist_3[i]<0 else 0 for i in range(len(dist_3))])

        all_dist = np.sqrt(dist_1 ** 2 + dist_2 ** 2 + dist_3 ** 2)

        sort_by_dist_idx = np.argsort(all_dist)
        #print(sort_by_dist_idx)
        sort_qed = np.array(qed)[sort_by_dist_idx]
        sort_logp = np.array(logp)[sort_by_dist_idx]
        sort_sas = np.array(sas)[sort_by_dist_idx]
        sort_dist = np.array(all_dist)[sort_by_dist_idx]
        sort_smi = np.array(smiles)[sort_by_dist_idx]

        min_qed.extend(list(sort_qed[:10]))
        min_logp.extend(list(sort_logp[:10]))
        min_sas.extend(list(sort_sas[:10]))
        min_dist.extend(list(sort_dist[:10]))
        min_smi.extend(list(sort_smi[:10]))
    
    final_sort_idx = np.argsort(min_dist)
    top_qed = np.array(min_qed)[final_sort_idx]
    top_logp = np.array(min_logp)[final_sort_idx]
    top_sas = np.array(min_sas)[final_sort_idx]
    top_dist = np.array(min_dist)[final_sort_idx]
    top_smi = np.array(min_smi)[final_sort_idx]
    #print(top_dist)

    return top_qed[:30], top_logp[:30], top_sas[:30], top_dist[:30], top_smi[:30]

In [6]:
def top_zinc_pareto_points():
    pwd = "/Users/nkusanda/Desktop/RESEARCH/olympus-ai4mat/DATA_3opt/"
    min_distances = []

    min_qed = []
    min_logp = []
    min_sas = []
    min_dist = []
    min_smi = []

    df = pd.read_csv(pwd + "zinc_pareto_tagged_logp_qed.csv")


    smiles = list(df['smiles'])
    logp = list(df['logP'])
    qed = list(df['qed'])
    sas = list(df['SAS'])
    pareto = list(df['pareto'])

    qed_pareto = [qed[i] for i in range(len(smiles)) if pareto[i] == 1]
    logp_pareto = [logp[i] for i in range(len(smiles)) if pareto[i] == 1]
    sas_pareto = [sas[i] for i in range(len(smiles)) if pareto[i] == 1]
    smiles_pareto = [smiles[i] for i in range(len(smiles)) if pareto[i] == 1]

    data = np.column_stack((qed_pareto,logp_pareto,sas_pareto))
    utopia = np.array([0.6,10,0.5])
    dist_1 = (utopia[0] - data[:,0]) / utopia[0]
    dist_2 = (utopia[1] - data[:,1]) / utopia[1]
    dist_3 = (utopia[2] - data[:,2]) / utopia[2]

    dist_1 = np.array([dist_1[i] if dist_1[i]>0 else 0 for i in range(len(dist_1))])
    dist_2 = np.array([dist_2[i] if dist_2[i]>0 else 0 for i in range(len(dist_2))])
    dist_3 = np.array([dist_3[i] if dist_3[i]<0 else 0 for i in range(len(dist_3))])

    all_dist = np.sqrt(dist_1 ** 2 + dist_2 ** 2 + dist_3 ** 2)

    sort_by_dist_idx = np.argsort(all_dist)
    #print(sort_by_dist_idx)
    sort_qed = np.array(qed_pareto)[sort_by_dist_idx]
    sort_logp = np.array(logp_pareto)[sort_by_dist_idx]
    sort_sas = np.array(sas_pareto)[sort_by_dist_idx]
    sort_dist = np.array(all_dist)[sort_by_dist_idx]
    sort_smi = np.array(smiles_pareto)[sort_by_dist_idx]

    min_qed.extend(list(sort_qed[:30]))
    min_logp.extend(list(sort_logp[:30]))
    min_sas.extend(list(sort_sas[:30]))
    min_dist.extend(list(sort_dist[:30]))
    min_smi.extend(list(sort_smi[:30]))
    

    return min_qed,min_logp,min_sas,min_dist,min_smi

In [7]:
# %%
chim_three = np.empty((4,30))
chim_three[0],chim_three[1],chim_three[2],chim_three[3],chim_smiles = top_min_all('chimera')
# %%
ctrl_three = np.empty((4,30))
ctrl_three[0],ctrl_three[1],ctrl_three[2],ctrl_three[3],ctrl_smiles = top_min_all('ctrl')
# %%
random_three = np.empty((4,30))
random_three[0],random_three[1],random_three[2],random_three[3],rand_smiles = top_min_all('random')
# %%
hv_three = np.empty((4,30))
hv_three[0],hv_three[1],hv_three[2],hv_three[3],hv_smiles = top_min_all('hv')
# %%
zinc_pareto = np.empty((4,30))
zinc_pareto[0],zinc_pareto[1],zinc_pareto[2],zinc_pareto[3],zinc_smi = top_zinc_pareto_points()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 11.39it/s]


### Blue: WeightedSum
### Orange: Random
### Green: Chimera
### Red: Hypervolume
### Purple: Zinc

In [ ]:
import plotly.graph_objects as go
fig = go.Figure(data=[go.Mesh3d(x=ctrl_three[0], y=ctrl_three[1], z=ctrl_three[2], color='#1f77b4', opacity=0.50)])
fig.add_trace(go.Mesh3d(x=random_three[0], y=random_three[1], z=random_three[2], color='#ff7f0e', opacity=0.50))
fig.add_trace(go.Mesh3d(x=chim_three[0], y=chim_three[1], z=chim_three[2], color='#2ca02c', opacity=0.50))
fig.add_trace(go.Mesh3d(x=hv_three[0], y=hv_three[1], z=hv_three[2], color='#d62728', opacity=0.50))
fig.add_trace(go.Mesh3d(x=zinc_pareto[0], y=zinc_pareto[1], z=zinc_pareto[2], color='#9467bd', opacity=0.50))

fig.update_layout(scene = dict(
                    xaxis_title='logp',
                    yaxis_title='qed',
                    zaxis_title='sas'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))
#fig.add_trace(go.Mesh3d(x=[0.6], y=[10], z=[0.5], color='#8c564b', opacity=0.50))

fig.show()

In [10]:
df = {}
df['qed'] = np.concatenate((ctrl_three[0],random_three[0],chim_three[0],hv_three[0]))
df['logp'] = np.concatenate((ctrl_three[1],random_three[1],chim_three[1],hv_three[1]))
df['sas'] = np.concatenate((ctrl_three[2],random_three[2],chim_three[2],hv_three[2]))
df['type'] = np.concatenate((['WeightedSum']*30,['Random']*30,['Chimera']*30,['Hypervolume']*30))
# %%
utop_df = {}
utop_df['qed'] = [0.6]
utop_df['logp'] = [10]
utop_df['sas'] = [0.5]
utop_df['type'] = ['Utopian point']
# %%
pareto_df = {}
pareto_df['qed'] = zinc_pareto[0]
pareto_df['logp'] = zinc_pareto[1]
pareto_df['sas'] = zinc_pareto[2]
pareto_df['type'] = ['Zinc'] * 30
# %%
ml_df = pd.DataFrame(df)
u_df = pd.DataFrame(utop_df)
p_df = pd.DataFrame(pareto_df)
# %%
all_df = pd.concat([ml_df,u_df,p_df])
fig = px.scatter_3d(all_df, x='logp', y='qed', z='sas', color='type',opacity = 1)
fig.show()